In [1]:
import pandas as pd

df_train = pd.read_csv('/home/ankur/Desktop/ML_DL_Projects/data/cmi-detect-behavior-with-sensor-data/train.csv')
df_train_demographics = pd.read_csv('/home/ankur/Desktop/ML_DL_Projects/data/cmi-detect-behavior-with-sensor-data/train_demographics.csv')
df_test = pd.read_csv('/home/ankur/Desktop/ML_DL_Projects/data/cmi-detect-behavior-with-sensor-data/test.csv')
df_test_demographics = pd.read_csv('/home/ankur/Desktop/ML_DL_Projects/data/cmi-detect-behavior-with-sensor-data/test_demographics.csv')
df_merged_train  = pd.merge(df_train, df_train_demographics, on='subject', how='left')
df_merged_test  = pd.merge(df_test, df_test_demographics, on='subject', how='left')

In [2]:
BFRB = [
    'Above ear - pull hair',
    'Forehead - pull hairline',
    'Forehead - scratch',
    'Eyebrow - pull hair',
    'Eyelash - pull hair',
    'Neck - pinch skin',
    'Neck - scratch',
    'Cheek - pinch skin'
]
# total count of values in df_merged_train['gesture']
print("Value counts:\n", df_merged_train['gesture'].value_counts())

# count where gesture is in BFRB
count_in_BFRB = df_merged_train['gesture'].isin(BFRB).sum()
print("\nRows in BFRB:", count_in_BFRB)

# count where gesture is NOT in BFRB
count_not_in_BFRB = (~df_merged_train['gesture'].isin(BFRB)).sum()
print("Rows NOT in BFRB:", count_not_in_BFRB)

# sanity check: should equal total row count
print("\nTotal rows:", len(df_merged_train))
print("Check:", count_in_BFRB + count_not_in_BFRB == len(df_merged_train))
for g in BFRB:
    df_merged_train[g] = (df_merged_train['gesture'] == g).astype(int)
df_merged_train["Non-Target"] = (
    df_merged_train["sequence_type"]
    .astype(str)
    .str.strip()
    .str.title()
    .map({"Target": 0, "Non-Target": 1})
)
df_merged_train.drop([
 'sequence_type', 
 'gesture',
 'row_id',   
 'subject',
 'orientation',
 'behavior',
 'phase'], axis=1, inplace=True)

df_merged_train = df_merged_train.interpolate(method="linear", axis=0)
nan_columns_X = df_merged_train.columns[df_merged_train.isnull().any()].tolist()
print("Columns in X with NaNs:\n", nan_columns_X)

df_merged_train.head()

Value counts:
 gesture
Text on phone                                 58462
Neck - scratch                                56619
Eyebrow - pull hair                           44305
Forehead - scratch                            40923
Forehead - pull hairline                      40802
Above ear - pull hair                         40560
Neck - pinch skin                             40507
Eyelash - pull hair                           40218
Cheek - pinch skin                            40124
Wave hello                                    34356
Write name in air                             31267
Pull air toward your face                     30743
Feel around in tray and pull out an object    17114
Glasses on/off                                13542
Drink from bottle/cup                         13093
Scratch knee/leg skin                         12328
Write name on leg                             10138
Pinch knee/leg skin                            9844
Name: count, dtype: int64

Rows in BFRB: 

/tmp/ipykernel_68126/397087711.py:43: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  df_merged_train = df_merged_train.interpolate(method="linear", axis=0)


Columns in X with NaNs:
 []


,sequence_id,sequence_counter,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,rot_z,thm_1,...,elbow_to_wrist_cm,Above ear - pull hair,Forehead - pull hairline,Forehead - scratch,Eyebrow - pull hair,Eyelash - pull hair,Neck - pinch skin,Neck - scratch,Cheek - pinch skin,Non-Target
0,SEQ_000007,0,6.683594,6.214844,3.355469,0.134399,-0.355164,-0.447327,-0.809753,28.943842,...,24.0,0,0,0,0,0,0,0,1,0
1,SEQ_000007,1,6.949219,6.214844,3.125000,0.143494,-0.340271,-0.428650,-0.824524,29.340816,...,24.0,0,0,0,0,0,0,0,1,0
2,SEQ_000007,2,5.722656,5.410156,5.421875,0.219055,-0.274231,-0.356934,-0.865662,30.339359,...,24.0,0,0,0,0,0,0,0,1,0
3,SEQ_000007,3,6.601562,3.531250,6.457031,0.297546,-0.264160,-0.238159,-0.885986,30.543730,...,24.0,0,0,0,0,0,0,0,1,0
4,SEQ_000007,4,5.566406,0.277344,9.632812,0.333557,-0.218628,-0.063538,-0.914856,29.317265,...,24.0,0,0,0,0,0,0,0,1,0


In [3]:
df_full_sensor_label_names =['sequence_id',
 'Above ear - pull hair',
 'Forehead - pull hairline',
 'Forehead - scratch',
 'Eyebrow - pull hair',
 'Eyelash - pull hair',
 'Neck - pinch skin',
 'Neck - scratch',
 'Cheek - pinch skin',
 'Non-Target']
df_full_sensor_target = df_merged_train[df_full_sensor_label_names]
df_full_sensor_target


,sequence_id,Above ear - pull hair,Forehead - pull hairline,Forehead - scratch,Eyebrow - pull hair,Eyelash - pull hair,Neck - pinch skin,Neck - scratch,Cheek - pinch skin,Non-Target
0,SEQ_000007,0,0,0,0,0,0,0,1,0
1,SEQ_000007,0,0,0,0,0,0,0,1,0
2,SEQ_000007,0,0,0,0,0,0,0,1,0
3,SEQ_000007,0,0,0,0,0,0,0,1,0
4,SEQ_000007,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
574940,SEQ_065531,0,0,0,0,0,0,0,0,1
574941,SEQ_065531,0,0,0,0,0,0,0,0,1
574942,SEQ_065531,0,0,0,0,0,0,0,0,1
574943,SEQ_065531,0,0,0,0,0,0,0,0,1


In [4]:
df_merged_train.drop([ 'Above ear - pull hair',
 'Forehead - pull hairline',
 'Forehead - scratch',
 'Eyebrow - pull hair',
 'Eyelash - pull hair',
 'Neck - pinch skin',
 'Neck - scratch',
 'Cheek - pinch skin',
 'Non-Target'], axis=1, inplace=True)

In [5]:
import numpy as np
import pandas as pd

def prepare_whole_sequences(df_x, df_y, max_steps=700):
    # Drop unnecessary columns early
    df_x = df_x.drop(columns=["sequence_counter"])  
    
    # Get number of features (exclude seq_id)
    feature_cols = [c for c in df_x.columns if c != "sequence_id"]
    n_features = len(feature_cols)
    
    # Find number of unique sequences
    sequence_ids = df_x["sequence_id"].unique()
    n_sequences = len(sequence_ids)

    # Preallocate final arrays
    X = np.zeros((n_sequences, max_steps, n_features), dtype=np.float32)
    Y = np.zeros((n_sequences, 9), dtype=np.float32)  # 9 classes, already multi-hot
    
    for i, seq_id in enumerate(sequence_ids):
        if i % 100 == 0:
            print(f"Processing {i}/{n_sequences}")

        # Extract sensor data for this sequence
        seq = df_x[df_x["sequence_id"] == seq_id][feature_cols].values.astype(np.float32)
        
        # Truncate if longer than max_steps
        seq = seq[:max_steps]
        
        # Fill into preallocated array
        X[i, :len(seq), :] = seq
        
        # Extract label row (drop sequence_id col, keep only class columns)
        label_row = df_y[df_y["sequence_id"] == seq_id].drop(columns=["sequence_id"]).iloc[0].values
        Y[i] = label_row
    
    return X, Y


In [6]:
x_train_fullsensor,y_train_fullsensor = prepare_whole_sequences(df_merged_train,df_full_sensor_target)
x_train_fullsensor.shape,x_train_fullsensor.shape

Processing 0/8151
Processing 100/8151
Processing 200/8151
Processing 300/8151
Processing 400/8151
Processing 500/8151
Processing 600/8151
Processing 700/8151
Processing 800/8151
Processing 900/8151
Processing 1000/8151
Processing 1100/8151
Processing 1200/8151
Processing 1300/8151
Processing 1400/8151
Processing 1500/8151
Processing 1600/8151
Processing 1700/8151
Processing 1800/8151
Processing 1900/8151
Processing 2000/8151
Processing 2100/8151
Processing 2200/8151
Processing 2300/8151
Processing 2400/8151
Processing 2500/8151
Processing 2600/8151
Processing 2700/8151
Processing 2800/8151
Processing 2900/8151
Processing 3000/8151
Processing 3100/8151
Processing 3200/8151
Processing 3300/8151
Processing 3400/8151
Processing 3500/8151
Processing 3600/8151
Processing 3700/8151
Processing 3800/8151
Processing 3900/8151
Processing 4000/8151
Processing 4100/8151
Processing 4200/8151
Processing 4300/8151
Processing 4400/8151
Processing 4500/8151
Processing 4600/8151
Processing 4700/8151
Proc

((8151, 700, 339), (8151, 700, 339))

In [7]:
del df_merged_train
del df_full_sensor_target
import gc
gc.collect() 

0

In [8]:
from sklearn.model_selection import train_test_split

# stratified split train+temp vs test
X_train, X_test, y_train, y_test = train_test_split(
    x_train_fullsensor, y_train_fullsensor, test_size=0.1, stratify=y_train_fullsensor, random_state=42
)

In [11]:
del x_train_fullsensor
del y_train_fullsensor
import gc
gc.collect() 

2169

In [12]:
x_train_all_sensor, x_val_all_sensor, y_train_all_sesnor, y_val_all_sensor = train_test_split(
    X_train, y_train, test_size=0.1, stratify=y_train, random_state=42
)
x_train_all_sensor.shape, x_val_all_sensor.shape, y_train_all_sesnor.shape, y_val_all_sensor.shape

((6601, 700, 339), (734, 700, 339), (6601, 9), (734, 9))

In [13]:
del X_train
del y_train
import gc
gc.collect() 

0

In [14]:
np.savez("train_val_test_pre_std.npz", x_train_all_sensor=x_train_all_sensor,y_train_all_sesnor=y_train_all_sesnor, x_val_all_sensor=x_val_all_sensor,y_val_all_sensor=y_val_all_sensor,x_test_all_sensors=X_test,y_test_all_sensors=y_test)

In [15]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Flatten (samples*timesteps, features)
X_train_flat = x_train_all_sensor.reshape(-1, x_train_all_sensor.shape[-1])
X_train_scaled = scaler.fit_transform(X_train_flat).reshape(x_train_all_sensor.shape)


In [16]:
del x_train_all_sensor
del X_train_flat
import gc
gc.collect() 

0

In [17]:
X_val_flat   = x_val_all_sensor.reshape(-1, x_val_all_sensor.shape[-1])
X_val_scaled   = scaler.transform(X_val_flat).reshape(x_val_all_sensor.shape)

X_test_flat  = X_test.reshape(-1, X_test.shape[-1])
X_test_scaled  = scaler.transform(X_test_flat).reshape(X_test.shape)

In [18]:
del x_val_all_sensor
del X_val_flat
del X_test
del X_test_flat
import gc
gc.collect() 

0

In [19]:
X_train_scaled.shape,y_train_all_sesnor.shape,X_val_scaled.shape,y_val_all_sensor.shape,X_test_scaled.shape,y_test.shape

((6601, 700, 339),
 (6601, 9),
 (734, 700, 339),
 (734, 9),
 (816, 700, 339),
 (816, 9))

In [21]:
np.savez("train_val_test_std.npz", X_train_scaled=X_train_scaled,y_train_all_sesnor=y_train_all_sesnor, X_val_scaled=X_val_scaled,y_val_all_sensor=y_val_all_sensor,X_test_scaled=X_test_scaled,y_test_all_sensors=y_test)

In [1]:
import numpy as np
import numpy as np

# Load the file
data = np.load("train_val_test_std.npz")

# Access arrays by the keys you saved
X_train_scaled     = data["X_train_scaled"]
y_train_all_sensor = data["y_train_all_sesnor"]   # watch spelling here
X_val_scaled       = data["X_val_scaled"]
y_val_all_sensor   = data["y_val_all_sensor"]
X_test_scaled      = data["X_test_scaled"]




In [2]:
y_test_all_sensors = data["y_test_all_sensors"]

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, GRU, MaxPooling1D, Input
from tensorflow.keras import backend as K
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')


num_classes = 9  # your 9 gesture classes

# Custom macro F1 score metric for multiclass
def f1_score(y_true, y_pred):
    y_pred = K.one_hot(K.argmax(y_pred, axis=-1), num_classes)  # convert to one-hot
    y_true = K.cast(y_true, "float32")

    tp = K.sum(K.cast(y_true * y_pred, 'float32'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float32'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float32'), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())

    return K.mean(f1)  # macro F1

# Define model
model = Sequential([
    Input(shape=(700, 339)),
    Conv1D(filters=96, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=256, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=256, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),
    GRU(32, return_sequences=False),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dense(32, activation='relu'),
    Dense(num_classes, activation='softmax',dtype='float32')  # multiclass classification
])

# Compile model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',  # one-hot labels
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),
        f1_score
    ]
)

model.summary()


2025-08-26 15:49:04.886483: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-26 15:49:04.986862: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756203545.048787   75797 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756203545.066297   75797 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756203545.158335   75797 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 696, 96)        │       162,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 348, 96)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 344, 256)       │       123,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 172, 256)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 168, 256)       │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 84, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 32)             │        27,840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 9)              │           297 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 650,377 (2.48 MB)

 Trainable params: 650,377 (2.48 MB)

 Non-trainable params: 0 (0.00 B)

In [4]:
tf.keras.backend.clear_session()

In [5]:
print("go ahead with training ")

go ahead with training 


In [ ]:
history = model.fit(
    X_train_scaled, 
    y_train_all_sensor,
    epochs=100,
    batch_size=10,
    validation_data=(X_val_scaled, y_val_all_sensor),  # use your validation set
    verbose=1
)
history

Epoch 1/100


2025-08-26 15:52:03.322517: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.
I0000 00:00:1756203723.590532   76040 cuda_dnn.cc:529] Loaded cuDNN version 90300


661/661 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3614 - f1_score: 0.0578 - loss: 2.0146 - precision: 0.2721 - recall: 0.0111